# Adir1.0 data downloads

This notebook provides information about how to download data from the [MalariaGEN Vector Observatory Asian Vector Genomic Surveillance Project](https://www.malariagen.net/project/vector-observatory-asia/), for *Anopheles dirus*. These data are the first release (v1.0), and include sample metadata, raw sequence reads, sequence read alignments, and single nucleotide polymorphism (SNP) calls. 

Code examples that are intended to be run via a Linux command line are prefixed with an exclamation mark (!). If you are running these commands directly from a terminal, remove the exclamation mark.

Examples in this notebook assume you are downloading data to a local folder within your home directory at the path `~/vo_adir_release/`. Change this if you want to download to a different folder on the local file system.

## Data hosting

`Adir1` data are hosted by several different services.

Raw sequence reads in FASTQ format and sequence read alignments in BAM format are hosted by the European Nucleotide Archive (ENA). This guide provides examples of downloading data from ENA via FTP using the `wget` command line tool, but please note that there are several other options for downloading data, see the [ENA documentation on how to download data files](https://ena-docs.readthedocs.io/en/latest/retrieval/file-download.html) for more information.  

SNP calls in VCF and Zarr formats are hosted on S3-compatible object storage at the Sanger Institute. This guide provides examples of downloading these data using `wget`.

Sample metadata in CSV format are hosted on Google Cloud Storage (GCS) in the `vo_adir_release_master_us_central1` bucket, which is a multi-region bucket located in the United States. All data hosted on GCS are publicly accessible but do require an authentication step, please see details on the [Vector Observatory Data Access page](https://malariagen.github.io/vector-data/vobs/vobs-data-access.html).

The guide below provides examples of downloading data from GCS to a local computer using the `wget` and `gsutil` command line tools. For more information about `gsutil`, see the [gsutil tool documentation](https://cloud.google.com/storage/docs/gsutil).

## Sample sets

Data in these releases are organised into sample sets. Each of these sample sets corresponds to a set of mosquito specimens contributed by a collaborating study. Depending on your objectives, you may want to download data from only specific sample sets, or all sample sets. For convenience there is a tab-delimited manifest file listing all sample sets in the release, this can be downloaded via `gsutil` to a directory on the local file system, e.g.:

In [ ]:
!mkdir -pv ~/vo_adir_release/v1.0/
!gsutil cp gs://vo_adir_release_master_us_central1/v1.0/manifest.tsv ~/vo_adir_release/v1.0/

Here are the file contents:

In [ ]:
!cat ~/vo_adir_release/v1.0/manifest.tsv

For more information about these sample sets, you can explore the [Adir1.0 data user guide](https://malariagen.github.io/vector-data/adir1/adir1.0.html).

## Sample metadata

Data about the samples that were sequenced to generate this data resource are available, including the time and place of collection, the gender of the specimen, and our call regarding the species of the specimen.

### Specimen collection metadata

Specimen collection metadata can be downloaded from GCS. E.g., sample metadata for all sample sets can be downloaded using `gsutil`. If you only want the sample metadata for a single sample set, these can be accessed by including the sample set name on the link below, e.g. to access the metadata for `1278-VO-TH-KOBYLINSKI-VMF00153`, you would use: `gs://vo_adir_release_master_us_central1/v1.0/metadata/general/1278-VO-TH-KOBYLINSKI-VMF00153`:

In [ ]:
!mkdir -pv ~/vo_adir_release/v1.0/metadata/1278-VO-TH-KOBYLINSKI-VMF00153/
!gsutil -m rsync -r gs://vo_adir_release_master_us_central1/v1.0/metadata/general/1278-VO-TH-KOBYLINSKI-VMF00153/ ~/vo_adir_release/v1.0/metadata/1278-VO-TH-KOBYLINSKI-VMF00153/

Here are the first few rows of the sample metadata for sample set `1278-VO-TH-KOBYLINSKI-VMF00153`:

In [ ]:
!head ~/vo_adir_release/v1.0/metadata/1278-VO-TH-KOBYLINSKI-VMF00153/samples.meta.csv

The `sample_id` column gives the sample identifier used throughout all analyses.

The `country`, `location`, `latitude` and `longitude` columns give the location where the specimen was collected.

The `year` and `month` columns give the approximate date when the specimen was collected.


## SNP calls (VCF format)

### SNP genotypes

SNP genotypes for individual mosquitoes in VCF format are available for download from Sanger S3-compatible object storage. A VCF file is available for each individual sample. To download a VCF file for a given sample, you will need the sample identifier and the sample set in which the sample belongs. Then inspect the data catalog in the metadata. E.g., for sample set `1278-VO-TH-KOBYLINSKI-VMF00153`:

In [ ]:
!head ~/vo_adir_release/v1.0/metadata/1278-VO-TH-KOBYLINSKI-VMF00153/wgs_snp_data.csv | cut -f1,4 -d,

A VCF file and associated tabix index can be downloaded via wget, e.g.:

In [ ]:
!wget --no-clobber https://1278-vo-th-kobylinski-vmf00153.cog.sanger.ac.uk/VBS45974-6296STDY9478582.vcf.gz
!wget --no-clobber https://1278-vo-th-kobylinski-vmf00153.cog.sanger.ac.uk/VBS45974-6296STDY9478582.vcf.gz.tbi

Note that each of these VCF files is around 3 Gb, so downloading may take some time, and sufficient local storage will be needed.

Each of these VCF files is an "all sites" VCF file, meaning that genotypes have been called at all genomic positions where the reference nucleotide is not "N", regardless of whether variation is observed in the given sample. This means that VCFs from multiple samples can be merged easily to create a multi-sample VCF, which may be required for certain analyses. For example, the code below merges VCFs for two samples for contig KB672490 up to 1 Mbp: 

In [ ]:
!bcftools merge --output-type z --regions KB672490:1-1000000 --output merged.vcf.gz VBS45974.vcf.gz VBS45974.vcf.gz 

If you are just interested in analysing variants within a given set of samples, you might like to filter the merged VCF to remove non-variant sites and alleles, e.g., using [bcftools view](http://samtools.github.io/bcftools/bcftools.html#view):

In [ ]:
!bcftools view --output-type z --output-file merged_variant.vcf.gz --min-ac 1:nonmajor --trim-alt-alleles merged.vcf.gz

### Site filters

SNP calling is not always reliable, and we have created some site filters to allow excluding low quality SNPs. For *An. funestus* and *An. gambiae*, these are available as a VCF file. For *An. dirus*, they are only available as a Zarr array (see below). If you would like to filter your VCF based on sites passing the filter, you will need to extract the data from the zarr array, and subset your VCF based on these locations (e.g. using bcftools --regions).

## SNP calls (Zarr format)

SNP data are also available in Zarr format, which can be convenient and efficient to use for certain types of analysis. These data can be analysed directly in the cloud without downloading to the local system, see the [Adir1 cloud data access guide](https://malariagen.github.io/vector-data/adir1/cloud.html) for more information. The data can also be downloaded to your own system for local analysis if that is more convenient. Below are examples of how to download the Zarr data to your local system.

The data are organised into several Zarr hierarchies. 

### SNP sites and alleles

Data on the genomic positions (sites) and reference and alternate alleles that were genotyped can be downloaded as follows:

In [ ]:
!mkdir -pv ~/vo_adir_release/v1.0/snp_genotypes/all/sites/
!gsutil -m rsync -r \
    gs://vo_adir_release_master_us_central1/v1.0/snp_genotypes/all/sites/ \
    ~/vo_adir_release/v1.0/snp_genotypes/all/sites/

### Site filters

SNP calling is not always reliable, and we have created some site filters to allow excluding low quality SNPs. To download site filters data in Zarr format:

In [ ]:
!mkdir -pv ~/vo_adir_release/v1.0/site_filters/sc_20250610/dirus/
!gsutil -m rsync -r \
    gs://vo_adir_release_master_us_central1/v1.0/site_filters/sc_20250610/dirus/ \
    ~/vo_adir_release/v1.0/site_filters/sc_20250610/dirus/

### SNP genotypes

SNP genotypes are available for each sample set separately. E.g., to download SNP genotypes in Zarr format for sample set `1278-VO-TH-KOBYLINSKI-VMF00153`, excluding some data you probably won't need:

In [ ]:
!mkdir -pv ~/vo_adir_release/v1.0/snp_genotypes/all/1278-VO-TH-KOBYLINSKI-VMF00153/
!gsutil -m rsync -r \
        -x '.*/calldata/(AD|GQ|MQ)/.*' \
        gs://vo_adir_release_master_us_central1/v1.0/snp_genotypes/all/1278-VO-TH-KOBYLINSKI-VMF00153/ \
        ~/vo_adir_release/v1.0/snp_genotypes/all/1278-VO-TH-KOBYLINSKI-VMF00153/

## Feedback and suggestions

If there are particular analyses you would like to run, or if you have other suggestions for useful documentation we could add to this site, we would love to know, please get in touch via the [malariagen/vector-data GitHub discussion board](https://github.com/malariagen/vector-data/discussions).